In [1]:
# Подключения и Setup
import panel as pn
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import numpy as np
from io import BytesIO

pn.extension()

instructions = pn.pane.Markdown('''
# Regression Analysis Service

1. Drag & drop your CSV/XLSX file into the widget below.  
2. Select the **target** variable and one or more **features**.  
3. Нажмите **Run regression** и получите summary модели + scatter-plots с линиями тренда.
''')

pn.Column(instructions).servable()


Column
    [0] Markdown(str)

In [2]:
# Интерфейс и логика + графики
file_input = pn.widgets.FileInput(accept='.csv,.xlsx', name='Drag & Drop file')
target_select = pn.widgets.Select(name='Target', options=[], disabled=True)
features_select = pn.widgets.MultiSelect(name='Features', options=[], disabled=True, size=6)
run_button = pn.widgets.Button(name='Run regression', button_type='primary', disabled=True)
output = pn.Column()

def on_file_change(event):
    if not file_input.value: 
        return
    buf = BytesIO(file_input.value)
    try:
        df = pd.read_csv(buf) if file_input.filename.lower().endswith('.csv') else pd.read_excel(buf)
    except Exception as e:
        output.objects = [pn.pane.Markdown(f" Read error: {e}")]
        return
    pn.state.df = df
    cols = list(df.columns)
    target_select.options = cols
    features_select.options = cols
    target_select.disabled = False
    features_select.disabled = False
    run_button.disabled = False
    output.objects = [pn.pane.Markdown(" File loaded. Select variables.")]

file_input.param.watch(on_file_change, 'value')

def on_run(event):
    df = getattr(pn.state, 'df', None)
    if df is None:
        output.objects = [pn.pane.Markdown(" Upload a file first.")]
        return
    if not target_select.value or not features_select.value:
        output.objects = [pn.pane.Markdown(" Select target and features.")]
        return

    y = df[target_select.value]
    X = df[list(features_select.value)]
    X = sm.add_constant(X)
    try:
        model = sm.OLS(y, X, missing='drop').fit()
    except Exception as e:
        output.objects = [pn.pane.Markdown(f" Model error: {e}")]
        return

    res = [pn.pane.Markdown("```\n" + model.summary().as_text() + "\n```")]
    for feat in features_select.value:
        fig, ax = plt.subplots()
        ax.scatter(df[feat], y, alpha=0.7)
        x_vals = np.linspace(df[feat].min(), df[feat].max(), 100)
        y_vals = model.params['const'] + model.params[feat] * x_vals
        ax.plot(x_vals, y_vals, 'r-')
        ax.set_xlabel(feat)
        ax.set_ylabel(target_select.value)
        res.append(pn.pane.Matplotlib(fig, sizing_mode='scale_width'))
        plt.close(fig)
    output.objects = res

run_button.on_click(on_run)

pn.Column(
    file_input,
    pn.Row(target_select, features_select),
    run_button,
    output,
    sizing_mode='stretch_width'
).servable()


Column(sizing_mode='stretch_width')
    [0] FileInput(accept='.csv,.xlsx', name='Drag & Drop file')
    [1] Row
        [0] Select(disabled=True, name='Target')
        [1] MultiSelect(disabled=True, name='Features', size=6)
    [2] Button(button_type='primary', disabled=True, name='Run regression')
    [3] Column()